In [ ]:
%cd ..
# %pip install .

%load_ext autoreload

%autoreload 2

In [ ]:
import datajoint as dj

dj.config["database.host"] = "at-database.ad.bcm.edu"

dj.config["database.user"] = "cpapadop"
dj.config["database.password"] = "cpapadop#123"

from neuropixel_pipeline.readers.labview import LabviewNeuropixelMeta
from neuropixel_pipeline.schemata import probe, ephys

from neuropixel_pipeline.utils import TODO

from pathlib import Path

## Probe

In [ ]:
probe.ProbeType.fill_neuropixel_probes()
probe.ProbeType()

In [ ]:
probe.Probe.insert1(dict(
    probe='19398116102',
    probe_type='neuropixels 1.0 - 3B',
    probe_comment="Used in maria's recordings"
), skip_duplicates=True)
probe.Probe()

# PreClustering

In [ ]:
# Need to get from this to accessible kilosort dir
key = {'animal_id': 29187,
      'session': 25,
      'scan_idx': 1}

experiment = dj.create_virtual_module('experiment', 'pipeline_experiment')

session_rel = experiment.Session() & key
session_rel.fetch1()

In [ ]:
ephys.Session.add_session(key)
ephys.Session()

In [ ]:
# Uses contents only, lookup table
ephys.AcquisitionSoftware()

In [ ]:
probe.Probe()

In [ ]:
session_dir = Path('/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16')

labview_metadata = LabviewNeuropixelMeta.from_h5(session_dir)

insertion_key = dict(session_id=1, insertion_number=0)

ephys.ProbeInsertion.insert1(dict(
    **insertion_key,
    probe=labview_metadata.serial_number,
), skip_duplicates=True)
ephys.ProbeInsertion()

In [ ]:
# manual_insert_data = TODO('manual insertion data')
# ephys.InsertionLocation.insert(dict())
ephys.InsertionLocation()

In [ ]:
ephys.EphysFile.insert1(dict(
    file_path=session_dir,
    acq_software="LabviewV1",
), skip_duplicates=True)
ephys.EphysRecording.populate()
ephys.EphysRecording()

In [ ]:
probe.ElectrodeConfig()

In [ ]:
probe.ElectrodeConfig.Electrode()

In [ ]:
# Mostly fully computed (from neuropixel data I think)
# ephys.LFP.populate()

# Clustering

In [ ]:
# lookup table
ephys.ClusteringMethod()

In [ ]:
settings = {
    'NchanTOT': 385,
    'fs': 30000,
    'nt': 61,
    'Th': 8,
    'spkTh': 8,
    'Th_detect': 9,
    'nwaves': 6,
    'nskip': 25,
    'nblocks': 5,
    'binning_depth': 5,
    'sig_interp': 20,
    'probe_name': 'neuropixPhase3B1_kilosortChanMap.mat',
}
settings['nt0min'] = int(20 * settings['nt']/61)
settings['NT'] = 2 * settings['fs']
settings['n_chan_bin'] = settings['NchanTOT']


# # lookup table, more commonly used (need a simple interface to insert into this occasionally)
ephys.ClusteringParamSet.fill(
    params=settings,
    clustering_method='kilosort4',
    description="default kilosort4 params",
    skip_duplicates=True
)

ephys.ClusteringParamSet().fetch1()

In [ ]:
# lookup table
ephys.ClusterQualityLabel()

In [ ]:
task_source_rel = (ephys.EphysRecording & insertion_key).proj() * (ephys.ClusteringParamSet() & {'paramset_idx': 1}).proj()
task_source_key = task_source_rel.fetch1()

task_source_key['clustering_output_dir'] = session_dir / 'spikes_kilosort4'
task_source_key['task_mode'] = 'load'
ephys.ClusteringTask.insert1(task_source_key, skip_duplicates=True)
ephys.ClusteringTask()

In [ ]:
ephys.Clustering.populate()
ephys.Clustering()

In [ ]:
# lookup table
ephys.CurationType()

In [ ]:
from datetime import datetime

curation_source_key = (ephys.Clustering() & task_source_key).proj().fetch1()
# print(curation_source_key)

ephys.Curation.insert1(dict(
    **curation_source_key,
    curation_id=0,
    curation_time=datetime.now(), # might not always be true
    curation_output_dir=curation_source_key['file_path'],
    quality_control=False,
    curation='no curation',
    curation_note='notes on the curation'
), skip_duplicates=True)

ephys.Curation()

In [22]:
ephys.CuratedClustering.populate()
ephys.CuratedClustering()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/christos/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_4855/2590846706.py", line 1, in <module>
    ephys.CuratedClustering.populate()
  File "/home/christos/.local/lib/python3.8/site-packages/datajoint/autopopulate.py", line 153, in populate
    make(dict(key))
  File "/home/christos/Repos/neuropixel-pipeline/neuropixel_pipeline/schemata/ephys.py", line 487, in make
    self.Unit.insert([{**key, **u} for u in units])
  File "/home/christos/.local/lib/python3.8/site-packages/datajoint/table.py", line 384, in insert
    self.connection.query(query, args=list(
  File "/home/christos/.local/lib/python3.8/site-packages/datajoint/connection.py", line 215, in query
    self._execute_query(cursor, query, args, cursor_class, suppress_warnings)
  File "/home/christos/.local/lib/python3.8/site-packages/datajoint/connection.py

In [ ]:
ephys.CuratedClustering.Unit()